In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import copy
import random
from mnist import MNIST


In [3]:
mndata = MNIST()

X, y = mndata.load_training()

In [4]:
for i in range(2):
    print(mndata.display(X[i]))
    print(y[i])


............................
............................
............................
............................
............................
.....................@@.....
.............@@@@@@.@@......
........@@@@@@@@@@..........
........@@@@@@..@@..........
...........@@@..............
............@...............
............@...............
.............@..............
.............@@.............
..............@@@...........
................@@..........
.................@@.........
.................@@@........
.................@@@........
..............@@@@@.........
............@@@@@@..........
..........@@@@@.............
........@@@@@...............
......@@@@@@................
.....@@@@...................
............................
............................
............................
5

............................
............................
............................
............................
.................@..........
...............@@@@@........
..........

In [5]:
X = np.array(X)

In [6]:
record_num, features_num = X.shape
print( record_num, features_num)

60000 784


In [7]:
def scale_data(X):
    X = X / 255
    
    return X

In [8]:
X = scale_data(X)

In [9]:
bias_f = np.ones((record_num, 1))
X = np.hstack((bias_f, X))

record_num, features_num = X.shape
print( record_num, features_num)

60000 785


In [10]:
record_num, features_num = X.shape
print( record_num, features_num)

60000 785


In [11]:
# ДАННЫЕ ГОТОВЫ

In [12]:
def dot_product(features, weights):
    inner_sum = np.sum(features * weights)
        
    return inner_sum

In [13]:
def my_sigmoid(features, weights):
    weighted_sum = dot_product(features, weights)
    return 1.0 / (1.0 + math.exp(-weighted_sum))

In [14]:
def init_weights(features_num, a, b):
    return np.array(a + (b - a) * np.random.random(features_num))

In [15]:
def num_gradient_st(loss_fun, predict_fun, X, y, model_weights, w_delta=0.01, lambd=1):
    i = np.random.randint(0, len(X) - 1)
    current_loss = loss_fun(predict_fun(X[i], model_weights), y[i], model_weights)
    weights_delta = model_weights[:]
    grad = []
    
    for coord in range(len(model_weights)):
        weights_delta[coord] += w_delta
        delta_loss = loss_fun(predict_fun(X[i], weights_delta), y[i], weights_delta)
        deriv = (delta_loss - current_loss) / w_delta
        grad.append(deriv)
        weights_delta[coord] -= w_delta
        
    return np.array(grad)

In [16]:
def gradient_descent(loss_fun, predict_fun, X, y, initial_weights, learning_rate, iter_num, verbose=True, lambd=1):
    model_weights = initial_weights[:]
    
    for counter in range(iter_num):
        grad = num_gradient_st(loss_fun, predict_fun, X, y, model_weights)
        model_weights -= learning_rate * grad
        #loss = loss_fun(X, y, model_weights)
        
        #if verbose:
        #    print("Iter: %i, loss_value: %f" % (counter, loss))
        
    return model_weights

In [17]:
def one_vs_all(y, value):
    y_new = []
    for i in range(len(y)):
        if y[i] == value:
            y_new.append(1)
        else:
            y_new.append(-1)
    return y_new

In [18]:
def sign(x):
    return 1 if x >= 0.0 else -1

In [19]:
def svm_predict(features, weights):
    score = dot_product(features, weights)
    
    return score

In [20]:
def my_hinge_loss(predicted_value, true_value, model_weights):
    result = np.sum(model_weights*model_weights)/10000 + max(0.0, 1.0 - predicted_value * true_value)
        
    return result

In [21]:
def one_vs_all(y, value):
    y_new = []
    for i in range(len(y)):
        if y[i] == value:
            y_new.append(1)
        else:
            y_new.append(-1)
    return y_new

In [22]:
def find_10_weights(X, y_bin, learning_rate=0.1, iter_num=100, gradient_descent=gradient_descent, lambd=1):
    weights_vector = []
    for yi in y_bin:
        initial_weights = init_weights(features_num, -0.5, 0.5)
        optimal_weights_i = gradient_descent(my_hinge_loss, svm_predict, X, yi, initial_weights, learning_rate, iter_num, lambd)
        weights_vector.append(optimal_weights_i)
    return weights_vector
        
        
    
    
    

In [23]:
def y_split(y):
    y_10 = []
    for i in range(10):
        yi = one_vs_all(y, i)
        y_10.append(yi)
    return np.array(y_10)

In [24]:
def my_bad_score(X, y_bin, weights_vector):
    error_all = []
    for ind, yi in enumerate(y_bin):
            error = 0 
            for i in range(len(X)):
                score = svm_predict(X[i], weights_vector[ind])
                if score >= 0:
                    predict_val = 1
                else:
                    predict_val = -1
                error += abs(yi[i]-predict_val)
            error_all.append(error/2)
    return error_all

In [25]:
def last_predict(X, weights_vector):
    predict_y = []
    for elem in X:
        score = []
        for weights in weights_vector:
            score.append(np.sum(elem*weights))
        score = np.array(score)
        i, = np.where(score == np.max(score))
        #if int(i) == 8:
         #   if score[9] / score[8] > 1/1.5:
          #      i = 9
        predict_y.append(int(i))
    return predict_y

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)


In [27]:
y_train_binary = y_split(y_train)

In [28]:
y_val_binary = y_split(y_val)

In [ ]:
%%time
opt_weights_vector20000 = find_10_weights(X_train, y_train_binary, iter_num = 10000, learning_rate=0.03)

In [ ]:
predicted_value_train = last_predict(X_train, opt_weights_vector20000)

In [ ]:
print(classification_report(np.array(y_train), predicted_value_train))

In [32]:
predicted_value_train = last_predict(X_train, opt_weights_vector20000)

In [33]:
print(classification_report(np.array(y_train), predicted_value_train))

             precision    recall  f1-score   support

          0       0.94      0.95      0.94      4765
          1       0.90      0.97      0.93      5369
          2       0.90      0.82      0.86      4764
          3       0.83      0.88      0.85      4894
          4       0.83      0.93      0.88      4685
          5       0.85      0.78      0.81      4311
          6       0.91      0.95      0.93      4731
          7       0.93      0.90      0.91      5000
          8       0.82      0.84      0.83      4679
          9       0.90      0.76      0.83      4802

avg / total       0.88      0.88      0.88     48000



In [31]:
predicted_value_train = last_predict(X_train, opt_weights_vector20000)

In [33]:
print(classification_report(np.array(y_train), predicted_value_train))

             precision    recall  f1-score   support

          0       0.95      0.95      0.95      4765
          1       0.98      0.91      0.94      5369
          2       0.85      0.90      0.87      4764
          3       0.96      0.73      0.83      4894
          4       0.91      0.88      0.89      4685
          5       0.71      0.88      0.79      4311
          6       0.93      0.94      0.94      4731
          7       0.95      0.87      0.91      5000
          8       0.76      0.86      0.81      4679
          9       0.83      0.86      0.85      4802

avg / total       0.89      0.88      0.88     48000



In [117]:
np.save('model_weights.txt', opt_weights_vector20000)

In [114]:
b = np.loadtxt('test1.txt')

In [103]:
b

array([[-2.30198353,  0.46365312,  0.33542215, ..., -0.19638215,
         0.35032174,  0.03170914],
       [-0.55117626, -0.24129331, -0.40981807, ...,  0.24658262,
        -0.21541419,  0.40791423],
       [-1.89931654,  0.33228351, -0.4914297 , ..., -0.17098675,
        -0.48407764,  0.42168246],
       ...,
       [-0.58152146,  0.38996292,  0.24682509, ...,  0.1825674 ,
        -0.06330745,  0.19567791],
       [-5.61400128, -0.44754549, -0.48912427, ...,  0.12740404,
        -0.43186924, -0.42864107],
       [-2.79995664, -0.36340402, -0.01748211, ...,  0.27442626,
         0.43028569,  0.35448041]])

In [95]:
opt_weights_vector20000

[array([-2.30198353e+00,  4.63653116e-01,  3.35422155e-01,  3.67834280e-02,
         3.10772069e-01, -3.08963857e-01,  1.68209109e-01, -3.85184857e-01,
         3.95422864e-01,  4.83116883e-01,  1.02778836e-01, -1.50214465e-01,
        -4.43528039e-01, -2.04189192e-01,  2.45460723e-01, -2.99360783e-02,
        -1.22913474e-01,  2.23260979e-01, -2.32618382e-01,  1.06576104e-01,
        -3.41793419e-01,  5.72763312e-02,  1.64629922e-01, -1.90380224e-01,
        -6.00564114e-02, -2.82611982e-01,  4.03226752e-01, -1.83997996e-01,
        -1.74338871e-02, -1.42861366e-01, -2.88495590e-01, -9.98668589e-02,
         6.72722652e-02,  4.27588831e-01,  3.96999330e-01, -2.29035061e-01,
         1.07798967e-01, -2.96995243e-02, -3.39224471e-02,  2.19715701e-01,
        -5.91321489e-02, -1.21797870e-01,  1.23822362e-01,  1.58905955e-01,
         1.57105098e-01, -1.22209513e-01,  4.43423722e-01, -2.57827594e-01,
         1.71441866e-04, -4.67912775e-01,  3.75890529e-01, -1.70263298e-01,
        -6.4

In [115]:
predicted_value_train = last_predict(X_val, b)

print(classification_report(np.array(y_val), predicted_value_train))

             precision    recall  f1-score   support

          0       0.94      0.96      0.95      1158
          1       0.98      0.91      0.94      1373
          2       0.85      0.90      0.88      1194
          3       0.95      0.70      0.81      1237
          4       0.88      0.90      0.89      1157
          5       0.71      0.87      0.78      1110
          6       0.95      0.93      0.94      1187
          7       0.95      0.87      0.91      1265
          8       0.75      0.84      0.79      1172
          9       0.85      0.87      0.86      1147

avg / total       0.88      0.88      0.88     12000

